In [3]:
!pip install transformers==4.40.1 accelerate==0.30.0 bitsandbytes==0.43.1 datasets==2.19.0 vllm==0.4.1 openai==1.25.1

  Using cached transformers-4.40.1-py3-none-any.whl.metadata (137 kB)
  Using cached accelerate-0.30.0-py3-none-any.whl.metadata (19 kB)
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl.metadata (2.2 kB)
  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
  Using cached vllm-0.4.1-cp310-cp310-manylinux1_x86_64.whl.metadata (8.9 kB)
  Using cached openai-1.25.1-py3-none-any.whl.metadata (21 kB)
  Using cached huggingface_hub-0.33.0-py3-none-any.whl.metadata (14 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached pyarrow-20.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached pyarrow_hotfix-0.7-py3-none-any.whl.metadata (3.6 kB)
  Using cached di

# 8.4절 실습: LLM 서빙 프레임워크

## 예제 8.1. 실습에 사용할 데이터셋 준비

In [1]:
import torch
from datasets import load_dataset

def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

dataset = load_dataset("shangrilar/ko_text2sql", "origin")['test']
dataset = dataset.to_pandas()

for idx, row in dataset.iterrows():
  prompt = make_prompt(row['context'], row['question'])
  dataset.loc[idx, 'prompt'] = prompt

## 예제 8.2. 모델과 토크나이저를 불러와 추론 파이프라인 준비

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "shangrilar/yi-ko-6b-text2sql"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_id)
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/9.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.28M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 예제 8.3. 배치 크기에 따른 추론 시간 확인

In [6]:
import time
for batch_size in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  hf_pipeline(dataset['prompt'].tolist(), max_new_tokens=128, batch_size=batch_size)
  print(f'{batch_size}: {time.time() - start_time}')

1: 184.0524230003357
2: 156.1557333469391
4: 101.59300518035889
8: 65.75109958648682
16: 41.8425235748291
32: 33.338934659957886


## 예제 8.4. vLLM 모델 불러오기

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [2]:
import torch
from vllm import LLM, SamplingParams

model_id = "shangrilar/yi-ko-6b-text2sql"
llm = LLM(model=model_id, dtype=torch.float16, max_model_len=1024)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 06-25 10:02:18 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='shangrilar/yi-ko-6b-text2sql', speculative_config=None, tokenizer='shangrilar/yi-ko-6b-text2sql', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-25 10:02:18 utils.py:608] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 06-25 10:02:19 selector.py:77] Cannot use FlashAttention backend because the flash_attn package is not found. Please install it for better performance.
INFO 06-25 10:02:19 selector.py:33] Using XFormers backend.
INFO 06-25 10:02:21 weight_utils.py:193] Using model weights format ['*.safetensors']
INFO 06-25 10:02:24 model_runner.py:173] Loading model weights took 11.5127 GB
INFO 06-25 10:02:25 gpu_executor.py:119] # GPU blocks: 30787, # CPU blocks: 4096
INFO 06-25 10:02:28 model_runner.py:976] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-25 10:02:28 model_runner.py:980] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing ea

## 예제 8.5. vLLM을 활용한 오프라인 추론 시간 측정

In [3]:
import time

for max_num_seqs in [1, 2, 4, 8, 16, 32]:
  start_time = time.time()
  llm.llm_engine.scheduler_config.max_num_seqs = max_num_seqs
  sampling_params = SamplingParams(temperature=1, top_p=1, max_tokens=128)
  outputs = llm.generate(dataset['prompt'].tolist(), sampling_params)
  print(f'{max_num_seqs}: {time.time() - start_time}')

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [01:22<00:00,  1.35it/s]


1: 83.05956482887268


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:43<00:00,  2.59it/s]


2: 43.20751428604126


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:24<00:00,  4.50it/s]


...

8: 14.680073022842407


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:10<00:00, 11.05it/s]


16: 10.173588275909424


Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 112/112 [00:08<00:00, 13.83it/s]

32: 8.135132312774658


## 예제 8.6. 온라인 서빙을 위한 vLLM API 서버 실행

### 안내
모델을 여러 번 GPU에 올리기 때문에 CUDA out of memory 에러가 발생할 수 있습니다. 그런 경우 구글 코랩의 런타임 > 세션 다시 시작 후 예제 코드를 실행해주세요.
예제 실행에 데이터셋이 필요한 경우 예제 8.1의 코드를 실행해주세요.

In [ ]:
!python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 1024

## 예제 8.7. 백그라운드에서 vLLM API 서버 실행하기

In [6]:
!nohup python -m vllm.entrypoints.openai.api_server \
--model shangrilar/yi-ko-6b-text2sql --host 127.0.0.1 --port 8888 --max-model-len 512 &

nohup: appending output to 'nohup.out'


## 예제 8.8. API 서버 실행 확인

In [9]:
!curl http://localhost:8888/v1/models

curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused


## 예제 8.9. API 요청

In [ ]:
import json

json_data = json.dumps(
    {"model": "shangrilar/yi-ko-6b-text2sql",
      "prompt": dataset.loc[0, "prompt"],
      "max_tokens": 128,
      "temperature": 1}
    )

!curl http://localhost:8888/v1/completions \
    -H "Content-Type: application/json" \
    -d '{json_data}'

## 예제 8.10. OpenAI 클라이언트를 사용한 API 요청

In [ ]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8888/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="shangrilar/yi-ko-6b-text2sql",
                                 prompt=dataset.loc[0, 'prompt'], max_tokens=128)
print("생성 결과:", completion.choices[0].text)